In [2]:
!pip install anthropic
import os
os.environ['ANTHROPIC_API_KEY'] = "YOUR_API_KEY"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.0/946.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [3]:
import anthropic

client = anthropic.Anthropic()

In [17]:
system_prompt = """
    You are a top-of-the-line symbolic translator for clinical trials.
    You were built to solve the problem that, while patient data is well-defined
    and regularly structured, clinical trial definitions are not.

    You take natural-language clinical trial definitions that contain patient
    inclusion and exclusion criteria and convert them into a well-defined set of
    symbols that a program will be able to use to quickly parse well-formatted
    patient data files and assign a score to each trial based on the inclusion
    and exclusion criteria for the given trial.

    Your responses will always take the following format, without any extraneous
    prepended nor appended text:

    {
      "response": "rules",
      "inclusion_criterium": [
        {
          "rule": {
                "type": "age",
                "min": 58,
                "max": 70
            },
            "weight": 1.0
          }
        },
        {
          ...
        }
      ]
      "exclusion_criterium": [
        {
          "rule": {
                "type": "gender",
                "gender": 1
            }
        },
        {
          ...
        }
      ]
    }

    Specifications:
    - Every response must include a list of inclusion rules and exclusions rules
    - Both types of rules will have a list of criterium for the program to consider
    - The exclusion criterium will be considered to always be mandatory (1.0 weight)
    - The inclusion criterium can be more lenient, and you will assess how important the inclusion criteria is by assigning a weight from 0.0 to 1.0, which the program will consider when assigning a score to each potential patient.
    - Below you will see the strict dictionary you must stick to. These are the only criteria the engine knows how to verify at the current moment. However, if you feel like you cannot deliver a sufficiently good ruleset given the current dictionary, you may include the "other" rule type, described below.

    Symbol Dictionary:
    age: used to specify an age criteria for a patient.
    {
      “type”: “age”,
      “min”: <int> [years] (optional),
      “max”: <int> [years] (optional)
    }

    gender: used to specify a gender criteria for a patient.
    {
      “type”: “gender”,
      “gender”: <int> [0 for male, 1 for female, 2 for either] (2 is equivalent to not making this a rule)
    }

    medications: used to find whether a patient is prescribed certain medications.
    {
      “type”: “medications”,
      “medications”: <list[str]> [the lowercase names of specific medications] (these will be substring matched against the list of patient medications)
    }

    Example of medications:
    drug
    potassium chloride
    d5w
    sodium chloride
    ns
    furosemide
    insulin
    iso-osmotic dextrose
    5% dextrose
    sw
    magnesium sulfate
    morphine sulfate
    acetaminophen
    heparin
    calcium gluconate

    preexisting_conditions: used to find whether a patient has had prior diagnoses or conditions.
    {
      “type”: “preexisting_conditions”,
      “icd9_codes: <list[str]> [the ICD-9 codes describing the relevant conditions]
    }


    other: **Not to be used unless deemed strictly necessary.** Will inform the developers to prioritize work on adding capability to the engine to parse a rule of this format.
    {
      "type": "other"
      ... [whatever arguments you think are appropriate]
    }



    Notes:
    - Your goal is to be thorough and accurate. Clinical trials have been designed by medical professionals like yourself, and often certain criteria are only alluded to or implied. You should use the intuition from your training data to be as thorough and accurate as possible.
    - If for any reason you recieve a request that is not obviously a clinical trial definition, simply respond with:
    {
      "response": "error",
      "message": "Invalid clinical trial definition."
    }
    """

In [20]:
message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0.2,
    system=system_prompt,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """
                    ID: NCT00744835
"Inclusion Criteria:\n\n1. The age of ≥ 18 years of age and ≤ 70 years of age, gender is not limited;\n2. Confirmed or suspected bacterial infection (refer to Appendix 5);\n3. Infection-related organ failure does not exceed 24 hours; organ failure is defined as circulation, (SOFA) ≥ 3 points in at least one organ or system of the respiratory, kidney, liver, coagulation and central nervous system;\n4. The time interval between the selection of the test drug and the test drug is not more than 8 hours;\n5. Infertility test for women of childbearing age;\n6. Childbearing age within six months without child care plan and agreed to take effective measures during the study of contraception;\n7. Patients or legal representatives signed informed consent.

\n\nExclusion Criteria:\n\n1. Pregnant or lactating women, or unable to take effective measures of contraception;\n2. Patients are expected to live less than 28 days due to basic diseases, such as poor control of malignant tumor, cardiac arrest in 30 days, and end-stage lung disease.\n3. The patient has the following chronic organ dysfunction or immunosuppression (based on the chronic health scoring assessment of the APACHE II score) : a) heart: New York heart association cardiac function IV; B) breathing: chronic obstructive, obstructive, or vascular lung disease can lead to severe restrictions on activities, i.e. the inability to go upstairs or to do housework; Or clear chronic hypoxia, CO2 retention, secondary real erythrocyte, severe pulmonary hypertension (\\> 40 mmHg) or respiratory muscle dependence; C) kidneys: receiving long-term dialysis; D) liver: liver cirrhosis confirmed by biopsy and clear portal hypertension; The upper digestive tract hemorrhage caused by portal hypertension; Or previous liver failure/hepatic encephalopathy/hepatic coma; E) of immune function: accept the treatment of impact resistance to infection, such as immune suppression therapy, chemotherapy, radiation therapy, or for a long time the recent use of high doses of hormones), or sickness impact resistance to infection, such as leukemia, lymphoma and AIDS);\n4. Solid organ or bone marrow transplantation;\n5. Plant survival status;\n6. The following conditions occurred within 4 weeks prior to infection: a) acute pulmonary embolism; B) transfusion response; C) acute coronary syndrome;\n7. Confirmed or highly suspected of acute infectious diseases such as viral hepatitis activity and active tuberculosis;\n8. Patients with sinus bradycardia (less than 60 per minute);\n9. Severe anemia (hemoglobin \\< 7.0 g/dL);\n10. Uncontrolled bleeding in the past 24 hours;\n11. Large area burns or chemical burns (III degree burns area \\> 30% BSA);\n12. The average arterial pressure was \\< 65 mmHg after adequate liquid resuscitation and vasoactive drug therapy.\n13. Acute myeloid hematopoiesis was characterized by a lack of severe granulocytes (ANC \\< 500 / mm3), or severe thrombocytopenia (\\< 20,000 / mm3);\n14. Allergic to the active ingredient or its auxiliary materials;\n15. The medication patients are using may severely affect the metabolism of the drug;\n16. Patients and (or) legal representatives have signed an unresuscitation (DNR), or decided to withdraw life support (withdraw) or restrict life support for the intensity (of the patient) and sign the informed consent form;\n17. Participated in clinical intervention test in 3 months;\n18. The subject is a researcher or his immediate family member, or may have improper informed consent;\n19. The attending physician considers it inappropriate for the patient to participate in this test.",
"""
                }
            ]
        }
    ]
)
print(message.content)

[TextBlock(text='{\n  "response": "rules",\n  "inclusion_criterium": [\n    {\n      "rule": {\n        "type": "age",\n        "min": 18,\n        "max": 70\n      },\n      "weight": 1.0\n    }\n  ],\n  "exclusion_criterium": [\n    {\n      "rule": {\n        "type": "preexisting_conditions",\n        "icd9_codes": ["140-239", "428.0", "490-496", "585.6", "571.5", "204-208", "042"]\n      }\n    },\n    {\n      "rule": {\n        "type": "medications",\n        "medications": ["immunosuppressant", "chemotherapy", "corticosteroid"]\n      }\n    },\n    {\n      "rule": {\n        "type": "preexisting_conditions",\n        "icd9_codes": ["415.1", "410", "070", "011"]\n      }\n    },\n    {\n      "rule": {\n        "type": "other",\n        "description": "Several complex medical conditions that cannot be easily encoded with current ruleset, including: severe anemia, uncontrolled bleeding, large area burns, severe hypotension, severe granulocytopenia, thrombocytopenia, and various 

In [21]:
import json

print(message.content[0].text)
rules = json.loads(message.content[0].text)
print(rules)

{
  "response": "rules",
  "inclusion_criterium": [
    {
      "rule": {
        "type": "age",
        "min": 18,
        "max": 70
      },
      "weight": 1.0
    }
  ],
  "exclusion_criterium": [
    {
      "rule": {
        "type": "preexisting_conditions",
        "icd9_codes": ["140-239", "428.0", "490-496", "585.6", "571.5", "204-208", "042"]
      }
    },
    {
      "rule": {
        "type": "medications",
        "medications": ["immunosuppressant", "chemotherapy", "corticosteroid"]
      }
    },
    {
      "rule": {
        "type": "preexisting_conditions",
        "icd9_codes": ["415.1", "410", "070", "011"]
      }
    },
    {
      "rule": {
        "type": "other",
        "description": "Several complex medical conditions that cannot be easily encoded with current ruleset, including: severe anemia, uncontrolled bleeding, large area burns, severe hypotension, severe granulocytopenia, thrombocytopenia, and various administrative exclusions"
      }
    }
  ]
}
{'re